In [1]:
!pip install --upgrade google-cloud-aiplatform
!pip install --upgrade google-cloud-storage
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 54.5 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.66.0
    Uninstalling google-cloud-aiplatform-1.66.0:
      Successfully uninstalled google-cloud-aiplatform-1.66.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


In [16]:
!pip install matplotlib
!pip install tqdm

In [17]:
from google.cloud import aiplatform
from google.cloud import storage
import base64
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [18]:
# Replace with your project and endpoint information
PROJECT_ID = 'lively-gravity-437417-b8'
REGION = 'us-central1'  # e.g., 'us-central1'
ENDPOINT_ID = '9056564028293251072'

In [19]:
aiplatform.init(project=PROJECT_ID, location=REGION)

# Create the Endpoint object
endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID)

In [30]:
# Step 5: Upload and load the video
VIDEO_FILE = '/output_video.mp4'  # Replace with your video filename

# Check if the video file exists
if not os.path.exists(VIDEO_FILE):
    raise FileNotFoundError(f"The video file {VIDEO_FILE} was not found.")

# Step 6: Prepare to write the output video
video_capture = cv2.VideoCapture(VIDEO_FILE)

fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

OUTPUT_VIDEO_FILE = 'output_video_od.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(OUTPUT_VIDEO_FILE, fourcc, fps, (frame_width, frame_height))

# Initialize a progress bar
pbar = tqdm(total=frame_count, desc='Processing Frames')

# Optional: Limit frame rate by processing every nth frame
frame_skip = 1  # Process every frame; increase this number to skip frames

frame_number = 0

while True:
    ret, frame = video_capture.read()
    if not ret:
        break  # End of video

    frame_number += 1

    if frame_number % frame_skip != 0:
        # Write the original frame to the output video without annotations
        video_writer.write(frame)
        pbar.update(1)
        continue

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Encode the frame as JPEG
    _, buffer = cv2.imencode('.jpg', frame_rgb)
    content = buffer.tobytes()

    # Encode in base64
    encoded_image = base64.b64encode(content).decode('utf-8')

    # Create the instance
    instance = {"content": encoded_image}
    instances = [instance]

    # Set the parameters (optional)
    parameters = {
        "confidenceThreshold": 0.5,
        "maxPredictions": 5
    }

    # Make the prediction
    response = endpoint.predict(instances=instances, parameters=parameters)

    # Extract predictions
    predictions = response.predictions[0]

    # Copy the frame for drawing
    annotated_frame = frame_rgb.copy()

    for i in range(len(predictions["displayNames"])):
        display_name = predictions["displayNames"][i]
        confidence = predictions["confidences"][i]
        bbox = predictions["bboxes"][i]  # [xMin, xMax, yMin, yMax]

        # Convert normalized coordinates to pixel values
        x_min = int(bbox[0] * frame_width)
        x_max = int(bbox[1] * frame_width)
        y_min = int(bbox[2] * frame_height)
        y_max = int(bbox[3] * frame_height)

        # Draw the bounding box
        cv2.rectangle(annotated_frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

        # Put the label and confidence
        label = f"{display_name}: {confidence:.2f}"
        cv2.putText(annotated_frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Convert back to BGR for writing to video
    annotated_frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

    # Write the frame to the output video
    video_writer.write(annotated_frame_bgr)

    # Update progress bar
    pbar.update(1)

# Release resources
video_capture.release()
video_writer.release()
pbar.close()



Processing Frames: 100%|██████████| 201/201 [01:18<00:00,  2.55it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# Step 8: Download the output video
from google.colab import files

files.download(OUTPUT_VIDEO_FILE)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>